<a href="https://colab.research.google.com/github/tristanengst/cnn-evolutionary-hyperparameters/blob/master/CNN_Playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Imports
import keras
import tensorflow as tf
from keras import layers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import random

#Presets
num_images = 4000

#Import data—data is local
data = pd.read_csv('train.csv')
images = data.iloc[0:num_images,1:]
train_images = images / 255.0
train_images = train_images.values.reshape(num_images, 28, 28, 1)
train_labels = data.iloc[0:num_images,:1]

train_images, validation_images, train_labels, validation_labels = train_test_split(train_images, train_labels, train_size = .5, random_state=42)

models = []

class Metadata:
    def __init__(self, dense_layers, dense_layer_nodes, kernels_0, kernels_1):
        self.dense_layers = dense_layers
        self.dense_layer_nodes = dense_layer_nodes
        self.kernels_0 = kernels_0
        self.kernels_1 = kernels_1
        
    def print_metadata(self):
        print(self.dense_layers, self.dense_layer_nodes, self.kernels_0, self.kernels_1)

class ModelWithMetadata:
    def get_accuracy(self, model):
        try:
            model.compile(optimizer=tf.train.AdamOptimizer(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
            model.fit(train_images, train_labels, batch_size=100, epochs=10, verbose=0, validation_data=(validation_images, validation_labels))
            return model.evaluate(validation_images, validation_labels)[1]
        except:
            print("Model failed")
            return 0.0
    
    def __init__(self, metadata):
        self.model = self.get_model_from_seed(metadata)
        self.metadata = metadata
        self.accuracy = self.get_accuracy(self.model)
        print("Trained model with accuracy ", self.accuracy)
        self.metadata.print_metadata()
    
    #Returns a model, input is of type metadata
    def get_model_from_seed(self, metadata):
        model = keras.Sequential()
        model.add(keras.layers.Conv2D(metadata.kernels_0, 2, strides=(1,1), padding="valid", input_shape=(28, 28, 1), activation="relu"))
        model.add(keras.layers.AveragePooling2D(pool_size=(2, 2), strides=None, padding='valid',))
        model.add(keras.layers.Conv2D(metadata.kernels_1, 2, strides=(1,1), padding="valid", input_shape=(28, 28, 1), activation="relu"))
        model.add(keras.layers.AveragePooling2D(pool_size=(2, 2), strides=None, padding='valid',))
        model.add(keras.layers.Flatten())
        for i in range(metadata.dense_layers):
            model.add(keras.layers.Dense(metadata.dense_layer_nodes[i], activation="relu", use_bias=True, kernel_initializer="glorot_uniform", bias_initializer="zeros"))
        model.add(keras.layers.Dense(10, activation="softmax", use_bias=True, kernel_initializer="glorot_uniform", bias_initializer="zeros"))
        return model

def mutate(model):
    dense_layers = 3
    dense_layer_nodes = list()
    for i in range(dense_layers):
        if (i < len(model.metadata.dense_layer_nodes)):
                dense_layer_nodes.append(max(10, model.metadata.dense_layer_nodes[i] + random.randint(-10,10)))
        else:
                dense_layer_nodes.append(random.randint(10,1001))
    kernels_0 = max(10, model.metadata.kernels_0 + random.randint(-10, 10))
    kernels_1 = max(10, model.metadata.kernels_1 + random.randint(-10, 10))
    return(Metadata(dense_layers, dense_layer_nodes, kernels_0, kernels_1))

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [0]:
def evolve(num_generations):
    for i in range(num_generations):
        generation_accuracy = 0
        for i in range(1):
            model = ModelWithMetadata(mutate(models[2]))
            models.append(model)
        for i in range(3):
            model = ModelWithMetadata(mutate(models[1]))
            models.append(model)
        for i in range(5):
            model = ModelWithMetadata(mutate(models[0]))
            models.append(model)
        models.sort(key=lambda model: model.accuracy)
        for model in models:
            generation_accuracy += model.accuracy / len(models)
        while (len(models) > 3):
            models.pop(0)
        print("Best accuracy of generation was", models[2].accuracy, "Average generational accuracy:", generation_accuracy)
        models[2].metadata.print_metadata()

models.append(ModelWithMetadata(Metadata(3, [1000, 1000, 200], 128, 32)))
models.append(ModelWithMetadata(Metadata(3, [1000, 500, 200], 128, 32)))
models.append(ModelWithMetadata(Metadata(3, [1000, 750, 200], 128, 64)))
evolve(40)
        
            

2000/2000 [==============================] - 1s 548us/step
Trained model with accuracy  0.9565
3 [1000, 1000, 200] 128 32
2000/2000 [==============================] - 1s 625us/step
Trained model with accuracy  0.956
3 [1000, 500, 200] 128 32
2000/2000 [==============================] - 2s 795us/step
Trained model with accuracy  0.9545
3 [1000, 750, 200] 128 64
2000/2000 [==============================] - 2s 985us/step
Trained model with accuracy  0.9515
3 [1009, 743, 210] 121 68
2000/2000 [==============================] - 2s 786us/step
Trained model with accuracy  0.95
3 [996, 492, 208] 134 27
2000/2000 [==============================] - 1s 733us/step
Trained model with accuracy  0.941
3 [1005, 490, 203] 125 23
2000/2000 [==============================] - 2s 760us/step
Trained model with accuracy  0.953
3 [1007, 509, 196] 137 27
2000/2000 [==============================] - 2s 878us/step
Trained model with accuracy  0.9595
3 [995, 997, 190] 133 36
2000/2000 [===========================